# Recortando todos os downloads pelo município de São Paulo

In [18]:
import rasterio
from rasterio import mask
from pathlib import Path
import numpy as np
import geopandas as gpd
import os

In [19]:
gdf_municipio = gpd.read_file('GIS/municipio-de-São-Paulo.gpkg')

In [20]:
gdf_municipio = gdf_municipio.to_crs('EPSG:32623')

In [21]:
i = 0
# anos = np.arange(1985, 2021, 5)

for y in np.arange(1985, 2021, 5):
    
    for path in Path(f'download/{y}').rglob('*'):
        
        if path.is_dir():
            
#             if i > 3:
#                 break
            
            print(f'Processando {path} ...')
#             i += 1
            
            if not os.path.exists(f'temp/{y}/{path.name}'):
                os.makedirs(f'temp/{y}/{path.name}')
            
            for cloud in Path(f'download/{y}/{path.name}').rglob('*BQA.TIF'):
                clouds = rasterio.open(f'download/{y}/{path.name}/{cloud.name}')
                
            for image in Path(f'download/{y}/{path.name}').rglob('*.TIF'):
                
                if image.name[-7:-4] == 'BQA':
                    print('BQA')
                    continue
                    
                dataset = rasterio.open(f'download/{y}/{path.name}/{image.name}')
                
                if dataset.res[0] < 30:
                    print(dataset.res)
                    continue
                    
                out_image = dataset.read(1)

                out_image[(clouds.read(1)[:,:] != 32) & \
                          (clouds.read(1)[:,:] != 672) & \
                          (clouds.read(1)[:,:] != 2720)] = 0
                out_meta = dataset.meta
                out_meta.update({"driver": "GTiff",
                                 "nodata": 0,
                                 "height": out_image.shape[0],
                                 "width": out_image.shape[1]})
                out_image = np.expand_dims(out_image, axis=0)
                
                print(out_image.shape)
                
                with rasterio.open(f'temp/{y}/{path.name}/{image.name}', "w", **out_meta) as dest:
                    dest.write(out_image)
                    
                result_dataset = rasterio.open(f'temp/{y}/{path.name}/{image.name}')
                    
                out_image_crop, out_transform_crop = \
                mask.mask(result_dataset, gdf_municipio.geometry, crop=True, invert=False)
                
                new_name = f'temp/{y}/{path.name}/{image.name.replace(".TIF", "-cropped.TIF")}'
                
                out_meta_crop = result_dataset.meta
                
                out_meta_crop.update({"driver": "GTiff",
                                     "nodata": 0,
                                     "height": out_image_crop.shape[1],
                                     "width": out_image_crop.shape[2],
                                     "transform": out_transform_crop})
                
                with rasterio.open(f'{new_name}', "w", **out_meta_crop) as dest:
                    dest.write(out_image_crop)
                    
                os.remove(f'temp/{y}/{path.name}/{image.name}')

Processando download/1985/LT05_L1TP_219077_19850616_20170219_01_T1 ...
BQA
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
Processando download/1985/LM05_L1TP_219077_19850515_20180406_01_T2 ...
(1, 3381, 3838)
BQA
(1, 3381, 3838)
(1, 3381, 3838)
(1, 3381, 3838)
Processando download/1985/LT05_L1TP_219077_19851022_20170218_01_T1 ...
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
(1, 6901, 7761)
BQA
(1, 6901, 7761)
(1, 6901, 7761)
Processando download/1985/LT05_L1TP_219076_19850531_20170219_01_T1 ...
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
BQA
Processando download/1985/LT05_L1TP_219076_19851022_20170218_01_T1 ...
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
(1, 6911, 7771)
BQA
Processando download/1985/LM05_L1TP_219077_19850819_20180407_01_T2 ...
(1, 3380, 3838)
(1, 3380, 3838)
(1, 3380, 

(1, 6981, 7781)
Processando download/1995/LT05_L1TP_219076_19950527_20170108_01_T1 ...
(1, 6981, 7771)
(1, 6981, 7771)
BQA
(1, 6981, 7771)
(1, 6981, 7771)
(1, 6981, 7771)
(1, 6981, 7771)
(1, 6981, 7771)
Processando download/1995/LT05_L1TP_219076_19951018_20170106_01_T1 ...
(1, 6981, 7751)
(1, 6981, 7751)
(1, 6981, 7751)
(1, 6981, 7751)
BQA
(1, 6981, 7751)
(1, 6981, 7751)
(1, 6981, 7751)
Processando download/2000/LE07_L1TP_219077_20000109_20170215_01_T1 ...
(1, 6961, 7841)
(15.0, 15.0)
(1, 6961, 7841)
(1, 6961, 7841)
(1, 6961, 7841)
(1, 6961, 7841)
(1, 6961, 7841)
(1, 6961, 7841)
BQA
(1, 6961, 7841)
Processando download/2000/LT05_L1TP_219076_20001116_20161213_01_T1 ...
(1, 7021, 7791)
(1, 7021, 7791)
BQA
(1, 7021, 7791)
(1, 7021, 7791)
(1, 7021, 7791)
(1, 7021, 7791)
(1, 7021, 7791)
Processando download/2000/LE07_L1TP_219077_20000617_20170211_01_T1 ...
(1, 6961, 7831)
(1, 6961, 7831)
(1, 6961, 7831)
(1, 6961, 7831)
(15.0, 15.0)
(1, 6961, 7831)
(1, 6961, 7831)
BQA
(1, 6961, 7831)
(1, 696

Processando download/2005/LT05_L1TP_219076_20050506_20161126_01_T1 ...
(1, 6941, 7891)
(1, 6941, 7891)
(1, 6941, 7891)
BQA
(1, 6941, 7891)
(1, 6941, 7891)
(1, 6941, 7891)
(1, 6941, 7891)
Processando download/2005/LT05_L1TP_219076_20051114_20161123_01_T1 ...
(1, 6951, 7841)
(1, 6951, 7841)
(1, 6951, 7841)
(1, 6951, 7841)
(1, 6951, 7841)
(1, 6951, 7841)
BQA
(1, 6951, 7841)
Processando download/2005/LT05_L1TP_219077_20050623_20161125_01_T1 ...
(1, 6941, 7871)
(1, 6941, 7871)
(1, 6941, 7871)
(1, 6941, 7871)
(1, 6941, 7871)
(1, 6941, 7871)
(1, 6941, 7871)
BQA
Processando download/2005/LT05_L1TP_219076_20050607_20161127_01_T1 ...
(1, 6941, 7891)
(1, 6941, 7891)
(1, 6941, 7891)
(1, 6941, 7891)
(1, 6941, 7891)
BQA
(1, 6941, 7891)
(1, 6941, 7891)
Processando download/2005/LT05_L1TP_219076_20050911_20161125_01_T1 ...
BQA
(1, 6941, 7901)
(1, 6941, 7901)
(1, 6941, 7901)
(1, 6941, 7901)
(1, 6941, 7901)
(1, 6941, 7901)
(1, 6941, 7901)
Processando download/2005/LT05_L1TP_219077_20050114_20161129_01_T

(1, 7711, 7611)
(1, 7711, 7611)
(1, 7711, 7611)
(1, 7711, 7611)
(1, 7711, 7611)
BQA
Processando download/2015/LC08_L1TP_219076_20151110_20170402_01_T1 ...
(1, 7721, 7621)
(1, 7721, 7621)
BQA
(1, 7721, 7621)
(1, 7721, 7621)
(1, 7721, 7621)
(1, 7721, 7621)
(1, 7721, 7621)
(1, 7721, 7621)
(15.0, 15.0)
(1, 7721, 7621)
(1, 7721, 7621)
Processando download/2015/LC08_L1TP_219076_20151025_20170402_01_T1 ...
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(15.0, 15.0)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
BQA
(1, 7711, 7621)
Processando download/2015/LC08_L1TP_219077_20150806_20170406_01_T1 ...
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(15.0, 15.0)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
BQA
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
Processando download/2015/LC08_L1TP_219077_20150603_20170408_01_T1 ...
(1, 7711, 7621)
(1, 7711, 7621)
(15.0, 15.0)
BQA
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 

(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
BQA
(15.0, 15.0)
Processando download/2020/LC08_L1TP_219076_20201209_20210313_01_T1 ...
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
BQA
(15.0, 15.0)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
Processando download/2020/LC08_L1TP_219076_20200616_20200626_01_T1 ...
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(15.0, 15.0)
(1, 7721, 7631)
(1, 7721, 7631)
BQA
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
(1, 7721, 7631)
Processando download/2020/LC08_L1TP_219077_20201209_20210313_01_T1 ...
(1, 7711, 7621)
BQA
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(15.0, 15.0)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
Processando download/2020/LC08_L1TP_219077_20200702_20200708_01_T1 ...
(15.0, 15.0)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7621)
(1, 7711, 7